In [44]:
import gc
import pickle
import gensim
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

In [45]:
def get_agg_features(dfs,f1,f2,agg,log):
    # [train_df,test_df],'user_id','','size',click_log
    # [train_df,test_df],'user_id','creative_id','unique',click_log
    #判定特殊情况
    if type(f1)==str:
        f1=[f1]
    if agg!='size':
        data=log[f1+[f2]]
    else:
        data=log[f1] 
    f_name='_'.join(f1)+"_"+f2+"_"+agg     
    #聚合操作    
    if agg=="size":
        tmp = pd.DataFrame(data.groupby(f1).size()).reset_index()
    elif agg=="count":
        tmp = pd.DataFrame(data.groupby(f1)[f2].count()).reset_index()
    elif agg=="mean":
        tmp = pd.DataFrame(data.groupby(f1)[f2].mean()).reset_index()
    elif agg=="unique":
        tmp = pd.DataFrame(data.groupby(f1)[f2].nunique()).reset_index()
    elif agg=="max":
        tmp = pd.DataFrame(data.groupby(f1)[f2].max()).reset_index()
    elif agg=="min":
        tmp = pd.DataFrame(data.groupby(f1)[f2].min()).reset_index()
    elif agg=="sum":
        tmp = pd.DataFrame(data.groupby(f1)[f2].sum()).reset_index()
    elif agg=="std":
        tmp = pd.DataFrame(data.groupby(f1)[f2].std()).reset_index()
    elif agg=="median":
        tmp = pd.DataFrame(data.groupby(f1)[f2].median()).reset_index()
    else:
        raise "agg error"   
    #赋值聚合特征
    for df in dfs:
        try:
            del df[f_name]
        except:
            pass
        tmp.columns = f1+[f_name]
        df[f_name]=df.merge(tmp, on=f1, how='left')[f_name] 
    del tmp
    del data
    gc.collect()
    return [f_name]


In [46]:
def sequence_text(dfs,f1,f2,log):
    f_name='sequence_text_'+f1+'_'+f2
    print(f_name)
    #遍历log，获得用户的点击序列
    dic,items={},[]
    for item in log[[f1,f2]].values:
        try:
            dic[item[0]].append(str(item[1]))
        except:
            dic[item[0]]=[str(item[1])]      
    for key in dic:
        items.append([key,' '.join(dic[key])])
    #赋值序列特征
    temp=pd.DataFrame(items)
    temp.columns=[f1,f_name]
    temp = temp.drop_duplicates(f1)
    for df in dfs:
        try:
            del df[f_name]
        except:
            pass
        temp.columns = [f1]+[f_name]
        df[f_name]=df.merge(temp, on=f1, how='left')[f_name]
    gc.collect() 
    del temp
    del items
    del dic
    return [f_name]

def kfold(train_df,test_df,log_data,pivot):
    #先对log做kflod统计，统计每条记录中pivot特征的性别年龄分布
    kfold_features=['age_{}'.format(i) for i in range(10)]+['gender_{}'.format(i) for i in range(2)]
    log=log_data[kfold_features+['user_id',pivot,'fold']]
    tmps=[]
    for fold in range(6):
        tmp = pd.DataFrame(log[(log['fold'] != fold) & (log['fold'] != 5)].groupby(pivot)[kfold_features].mean()).reset_index()
        tmp.columns=[pivot]+kfold_features
        tmp['fold']=fold
        tmps.append(tmp)
    tmp=pd.concat(tmps,axis=0).reset_index()
    tmp=log[['user_id',pivot,'fold']].merge(tmp,on=[pivot,'fold'],how='left')
    del log
    del tmps
    gc.collect() 
    #获得用户点击的所有记录的平均性别年龄分布
    tmp_mean = pd.DataFrame(tmp.groupby('user_id')[kfold_features].mean()).reset_index()
    tmp_mean.columns=['user_id']+[f+'_'+pivot+'_mean' for f in kfold_features]
    for df in [train_df,test_df]:
        temp=df.merge(tmp_mean,on='user_id',how='left')
        temp=temp.fillna(-1)
        for f1 in [f+'_'+pivot+'_mean' for f in kfold_features]:
            df[f1]=temp[f1]
        del temp
        gc.collect()
    del tmp
    del tmp_mean
    gc.collect()



def kfold_sequence(train_df,test_df,log_data,pivot): 
    #先对log做kflod统计，统计每条记录中pivot特征的性别年龄分布
    kfold_features=['age_{}'.format(i) for i in range(10)]+['gender_{}'.format(i) for i in range(2)]
    log=log_data[kfold_features+[pivot,'fold','user_id']]
    tmps=[]
    for fold in range(6):
        tmp = pd.DataFrame(log[(log['fold'] != fold) & (log['fold'] != 5)].groupby(pivot)[kfold_features].mean()).reset_index()
        tmp.columns=[pivot]+kfold_features
        tmp['fold']=fold
        tmps.append(tmp)
    tmp=pd.concat(tmps,axis=0).reset_index()
    tmp=log[[pivot,'fold','user_id']].merge(tmp,on=[pivot,'fold'],how='left')
    tmp=tmp.fillna(-1)   
    tmp[pivot+'_fold']=tmp[pivot]*10+tmp['fold']   
    del log
    del tmps
    gc.collect() 
    #获得用户点击记录的年龄性别分布序列
    tmp[pivot+'_fold']=tmp[pivot+'_fold'].astype(int)
    kfold_sequence_features=sequence_text([train_df,test_df],'user_id',pivot+'_fold',tmp)
    tmp=tmp.drop_duplicates([pivot+'_fold']).reset_index(drop=True)
    #对每条记录年龄性别分布进行标准化
    kfold_features=['age_{}'.format(i) for i in range(10)]+['gender_{}'.format(i) for i in range(2)]
    ss=StandardScaler()
    ss.fit(tmp[kfold_features])
    tmp[kfold_features]=ss.transform(tmp[kfold_features])
    for f in kfold_features:
        tmp[f]=tmp[f].apply(lambda x:round(x,4))   
    #将每条记录年龄性别分布转成w2v形式的文件
    with open('data/sequence_text_user_id_'+pivot+'_fold'+".{}d".format(12),'w') as f:
        f.write(str(len(tmp))+' '+'12'+'\n')
        for item in tmp[[pivot+'_fold']+kfold_features].values:
            f.write(' '.join([str(int(item[0]))]+[str(x) for x in item[1:]])+'\n') 
    tmp=gensim.models.KeyedVectors.load_word2vec_format('data/sequence_text_user_id_'+pivot+'_fold'+".{}d".format(12),binary=False)
    pickle.dump(tmp,open('data/sequence_text_user_id_'+pivot+'_fold'+".{}d".format(12),'wb'))
    del tmp
    gc.collect()  
    return kfold_sequence_features

In [47]:
click_log=pd.read_pickle('/Users/hesu/Documents/KT/Rank1st-master/data/click.pkl')
train_df=pd.read_pickle('/Users/hesu/Documents/KT/Rank1st-master/data/train_user.pkl')
test_df=pd.read_pickle('/Users/hesu/Documents/KT/Rank1st-master/data/test_user.pkl')

In [48]:
click_log.head(10)

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,age,...,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,gender_0,gender_1
0,1,3915646,22854,1,23627,1451,2,12180,250,-1,...,0,0,0,0,0,0,0,0,0,0
1,1,3920282,22182,1,23031,-1,12,10989,6,-1,...,0,0,0,0,0,0,0,0,0,0
2,1,410651,117365,1,106070,154,2,14002,319,2,...,1,0,0,0,0,0,0,0,1,0
3,1,3005387,82002,1,75201,10559,6,6886,154,-1,...,0,0,0,0,0,0,0,0,0,0
4,1,3702919,161988,1,145347,1989,2,24809,247,-1,...,0,0,0,0,0,0,0,0,0,0
5,1,3390855,11675,1,13097,1034,2,15361,320,-1,...,0,0,0,0,0,0,0,0,0,0
6,1,34263,193145,1,173339,-1,18,19748,73,3,...,0,1,0,0,0,0,0,0,1,0
7,1,3500918,130695,1,117489,-1,18,35662,322,-1,...,0,0,0,0,0,0,0,0,0,0
8,1,3460324,158399,1,142041,-1,5,21421,148,-1,...,0,0,0,0,0,0,0,0,0,0
9,1,3736271,206553,1,185579,2398,2,28579,248,-1,...,0,0,0,0,0,0,0,0,0,0


In [49]:
train_df.head(10)

,user_id,age,gender
0,1,3,0
1,2,9,0
2,3,6,1
3,4,4,0
4,5,3,0
5,6,5,0
6,7,5,1
7,8,4,0
8,9,4,0
9,10,8,1


In [50]:
test_df.head(10)

,user_id,age,gender
0,3000001,-1,-1
1,3000002,-1,-1
2,3000003,-1,-1
3,3000004,-1,-1
4,3000005,-1,-1
5,3000006,-1,-1
6,3000007,-1,-1
7,3000008,-1,-1
8,3000009,-1,-1
9,3000010,-1,-1


In [51]:
agg_features = []

In [52]:
agg_features+=get_agg_features([train_df,test_df],'user_id','','size',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','ad_id','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','creative_id','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','advertiser_id','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','industry','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','product_id','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','time','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','click_times','sum',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','click_times','mean',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','click_times','std',click_log)

In [53]:
train_df[agg_features]=train_df[agg_features].fillna(-1)
test_df[agg_features]=test_df[agg_features].fillna(-1)

In [54]:
train_df.head(10)

,user_id,age,gender,user_id__size,user_id_ad_id_unique,user_id_creative_id_unique,user_id_advertiser_id_unique,user_id_industry_unique,user_id_product_id_unique,user_id_time_unique,user_id_click_times_sum,user_id_click_times_mean,user_id_click_times_std
0,1,3,0,13,12,12,12,9,6,10,14,1.076923,0.277350
1,2,9,0,45,42,42,36,15,20,28,46,1.022222,0.149071
2,3,6,1,30,30,30,28,8,17,23,30,1.000000,0.000000
3,4,4,0,29,29,29,26,10,18,15,29,1.000000,0.000000
4,5,3,0,33,33,33,30,18,7,26,34,1.030303,0.174078
5,6,5,0,17,16,16,13,10,6,14,17,1.000000,0.000000
6,7,5,1,10,10,10,10,8,5,10,10,1.000000,0.000000
7,8,4,0,49,48,48,37,20,25,34,52,1.061224,0.242226
8,9,4,0,20,17,17,15,5,12,17,21,1.050000,0.223607
9,10,8,1,10,9,9,8,5,5,10,10,1.000000,0.000000


In [55]:
test_df.head(10)

,user_id,age,gender,user_id__size,user_id_ad_id_unique,user_id_creative_id_unique,user_id_advertiser_id_unique,user_id_industry_unique,user_id_product_id_unique,user_id_time_unique,user_id_click_times_sum,user_id_click_times_mean,user_id_click_times_std
0,3000001,-1,-1,11,11,11,10,5,7,7,11,1.000000,0.000000
1,3000002,-1,-1,61,49,49,41,21,24,37,65,1.065574,0.249590
2,3000003,-1,-1,36,35,35,29,17,15,23,36,1.000000,0.000000
3,3000004,-1,-1,20,17,17,14,9,7,15,20,1.000000,0.000000
4,3000005,-1,-1,28,23,23,22,12,11,24,29,1.035714,0.188982
5,3000006,-1,-1,16,16,16,14,8,11,12,21,1.312500,0.793200
6,3000007,-1,-1,10,10,10,10,5,8,8,10,1.000000,0.000000
7,3000008,-1,-1,28,25,26,21,14,12,24,31,1.107143,0.416270
8,3000009,-1,-1,24,23,23,22,12,18,19,25,1.041667,0.204124
9,3000010,-1,-1,28,26,26,26,10,19,20,29,1.035714,0.188982


In [56]:
text_features=[]
text_features+=sequence_text([train_df,test_df],'user_id','ad_id',click_log)
text_features+=sequence_text([train_df,test_df],'user_id','creative_id',click_log)
text_features+=sequence_text([train_df,test_df],'user_id','advertiser_id',click_log)
text_features+=sequence_text([train_df,test_df],'user_id','product_id',click_log)
text_features+=sequence_text([train_df,test_df],'user_id','industry',click_log)
text_features+=sequence_text([train_df,test_df],'user_id','product_category',click_log)
text_features+=sequence_text([train_df,test_df],'user_id','time',click_log)
text_features+=sequence_text([train_df,test_df],'user_id','click_times',click_log)

sequence_text_user_id_ad_id
sequence_text_user_id_creative_id
sequence_text_user_id_advertiser_id
sequence_text_user_id_product_id
sequence_text_user_id_industry
sequence_text_user_id_product_category
sequence_text_user_id_time
sequence_text_user_id_click_times


In [57]:
print(text_features)

['sequence_text_user_id_ad_id', 'sequence_text_user_id_creative_id', 'sequence_text_user_id_advertiser_id', 'sequence_text_user_id_product_id', 'sequence_text_user_id_industry', 'sequence_text_user_id_product_category', 'sequence_text_user_id_time', 'sequence_text_user_id_click_times']


In [58]:
train_df.head(10)

,user_id,age,gender,user_id__size,user_id_ad_id_unique,user_id_creative_id_unique,user_id_advertiser_id_unique,user_id_industry_unique,user_id_product_id_unique,user_id_time_unique,...,user_id_click_times_mean,user_id_click_times_std,sequence_text_user_id_ad_id,sequence_text_user_id_creative_id,sequence_text_user_id_advertiser_id,sequence_text_user_id_product_id,sequence_text_user_id_industry,sequence_text_user_id_product_category,sequence_text_user_id_time,sequence_text_user_id_click_times
0,1,3,0,13,12,12,12,9,6,10,...,1.076923,0.277350,724607 188507 773445 1458878 109959 66210 1678...,821396 209778 877468 1683713 122032 71691 1940...,7293 9702 29455 14668 11411 14681 17189 367 44...,-1 136 -1 -1 1334 -1 -1 -1 -1 1454 64 64 1261,326 6 106 326 -1 326 73 217 64 238 245 245 6,5 2 5 5 2 18 5 5 18 2 2 2 2,20 20 20 39 40 43 46 52 60 64 64 73 76,1 1 1 1 1 1 1 1 1 1 1 1 2
1,2,9,0,45,42,42,36,15,20,28,...,1.022222,0.149071,58788 139702 38066 541125 1107111 392680 14495...,63441 155822 39714 609050 1266180 441462 13069...,22885 10686 18562 25932 34505 22885 768 26006 ...,87 80 129 129 -1 87 1400 -1 1261 -1 111 129 -1...,318 238 6 6 47 318 317 47 6 47 242 6 6 47 47 4...,2 2 2 2 18 2 2 18 2 18 2 2 18 18 18 18 2 18 2 ...,10 11 14 17 28 28 28 38 38 39 41 42 42 42 44 4...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
2,3,6,1,30,30,30,28,8,17,23,...,1.000000,0.000000,586668 713448 629278 728308 527601 643108 3502...,661347 808612 710859 825434 593522 726940 3920...,32974 9877 18492 14186 17018 9058 8371 2336 39...,36256 40905 1674 35985 1674 -1 -1 1031 1786 22...,-1 -1 322 -1 322 6 54 6 322 322 205 302 322 32...,17 17 2 17 2 18 18 2 2 2 18 12 2 2 18 18 8 4 2...,12 13 14 14 14 17 19 22 31 36 37 44 47 47 50 5...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
3,4,4,0,29,29,29,26,10,18,15,...,1.000000,0.000000,37966 524312 511235 1638619 1698206 30773 1076...,39588 589886 574787 1892854 1962706 31070 1230...,19451 7976 13084 12130 23664 13299 10172 811 1...,1862 -1 2625 38743 41265 39422 39904 37758 -1 ...,238 25 248 -1 -1 -1 -1 -1 88 319 238 6 319 238...,2 18 2 17 17 17 17 4 18 2 2 2 2 2 2 2 2 5 2 2 ...,8 15 41 44 48 48 48 48 49 52 58 58 59 61 62 62...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
4,5,3,0,33,33,33,30,18,7,26,...,1.030303,0.174078,265971 314795 24966 41148 751113 925792 113378...,296145 350759 24333 43235 852327 1054434 12964...,11882 992 22885 9706 38760 2862 17745 31552 22...,-1 -1 87 136 -1 136 -1 -1 -1 1064 -1 2620 -1 -...,297 -1 318 6 322 6 288 322 319 238 322 322 203...,5 8 2 2 18 2 5 18 18 2 18 2 5 5 2 18 2 18 18 5...,3 13 14 15 20 21 24 25 27 28 29 30 32 32 35 35...,1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
5,6,5,0,17,16,16,13,10,6,14,...,1.000000,0.000000,1111812 1176804 1154489 589761 1722746 1842655...,1271602 1346445 1320331 664972 1991587 2133563...,7115 20441 39260 28422 28879 37156 25144 7115 ...,643 -1 29664 8495 -1 -1 -1 643 -1 -1 643 -1 -1...,6 203 242 183 9 13 54 6 54 54 6 54 54 293 247 ...,3 18 3 3 18 18 18 3 18 18 3 18 18 18 3 3 3,25 29 32 32 43 45 45 52 53 54 56 58 58 60 61 8...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
6,7,5,1,10,10,10,10,8,5,10,...,1.000000,0.000000,367016 365156 1788664 8466 575608 2267712 2342...,411126 409181 2069959 7054 648402 2634737 2723...,8921 27144 14734 2421 18103 30481 55274 22885 ...,-1 -1 -1 -1 -1 125 -1 87 23 1367,28 54 221 1 54 54 6 318 319 241,18 18 5 18 12 2 5 2 2 2,6 11 49 50 53 55 57 79 85 88,1 1 1 1 1 1 1 1 1 1
7,8,4,0,49,48,48,37,20,25,34,...,1.061224,0.242226,75704 518362 16404 107703 827392 59032 16559 1...,82560 582835 14993 119202 939476 63726 15155 1...,25896 32762 1903 8385 38428 24721 1295 11896 4...,2065 2065 -1 -1 1064 1469 1268 -1 1469 1751 -1...,238 238 1 6 242 21 5 297 21 238 6 86 230 60 86...,2 2 18 18 2 2 2 5 2 2 18 8 2 3 8 2 2 2 5 3 5 2...,10 10 11 14 19 26 28 28 28 30 31 33 34 43 45 4...,1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
8,9,4,0,20,17,17,15,5,12,17,...,1.050000,0.223607,265885 674122 674122 722775 80848 794984 10656...,2960

In [59]:
train_df.columns

Index(['user_id', 'age', 'gender', 'user_id__size', 'user_id_ad_id_unique',
       'user_id_creative_id_unique', 'user_id_advertiser_id_unique',
       'user_id_industry_unique', 'user_id_product_id_unique',
       'user_id_time_unique', 'user_id_click_times_sum',
       'user_id_click_times_mean', 'user_id_click_times_std',
       'sequence_text_user_id_ad_id', 'sequence_text_user_id_creative_id',
       'sequence_text_user_id_advertiser_id',
       'sequence_text_user_id_product_id', 'sequence_text_user_id_industry',
       'sequence_text_user_id_product_category', 'sequence_text_user_id_time',
       'sequence_text_user_id_click_times'],
      dtype='object')

In [60]:
train_df[['sequence_text_user_id_ad_id','sequence_text_user_id_creative_id']]

,sequence_text_user_id_ad_id,sequence_text_user_id_creative_id
0,724607 188507 773445 1458878 109959 66210 1678...,821396 209778 877468 1683713 122032 71691 1940...
1,58788 139702 38066 541125 1107111 392680 14495...,63441 155822 39714 609050 1266180 441462 13069...
2,586668 713448 629278 728308 527601 643108 3502...,661347 808612 710859 825434 593522 726940 3920...
3,37966 524312 511235 1638619 1698206 30773 1076...,39588 589886 574787 1892854 1962706 31070 1230...
4,265971 314795 24966 41148 751113 925792 113378...,296145 350759 24333 43235 852327 1054434 12964...
...,...,...
899995,84196 581202 827341 1286930 1270969 820416 157...,92193 654526 939409 1477926 1458800 931537 181...
899996,63976 30156 8817 55948 2410684 2589756 24966 1...,69204 30210 7400 60330 2804147 3011222 24333 1...
899997,575608 1094938 1197711 1985998 2506442 1540603...,648402 1251649 1370659 2302153 2915599 1779124...
899998,276030 110687 573549 14271 684333 532988 11263...,307407 122834 646020 12838 774777 599699 12879...


In [61]:
log=click_log.drop_duplicates(['user_id','creative_id']).reset_index(drop=True)


In [62]:
del click_log
gc.collect()

20

In [63]:
log['cont']=1
train_df['fold']=train_df.index%5
test_df['fold']=5
df=train_df.append(test_df)[['user_id','fold']].reset_index(drop=True)
log=log.merge(df,on='user_id',how='left')
del df
gc.collect()
#获取用户点击某特征的年龄性别平均分布
for pivot in ['creative_id','ad_id','product_id','advertiser_id','industry']:
    print("Kfold",pivot)
    kfold(train_df,test_df,log,pivot)
del log
gc.collect()       

Kfold creative_id
Kfold ad_id
Kfold product_id
Kfold advertiser_id
Kfold industry


0

In [64]:
click_log=pd.read_pickle('data/click.pkl')
log=click_log.reset_index(drop=True)
del click_log
gc.collect()
log['cont']=1
train_df['fold']=train_df.index%5
train_df['fold']=train_df['fold'].astype(int)
test_df['fold']=5
df=train_df.append(test_df)[['user_id','fold']].reset_index(drop=True)
log=log.merge(df,on='user_id',how='left')
#获取用户点击某特征的年龄性别分布序列
kfold_sequence_features=[] 
for pivot in ['creative_id','ad_id','product_id','advertiser_id','industry']:
    print("Kfold sequence",pivot)
    kfold_sequence_features+=kfold_sequence(train_df,test_df,log,pivot) 
del log
gc.collect()        

FileNotFoundError: [Errno 2] No such file or directory: 'data/click.pkl'